<a href="https://www.kaggle.com/code/mikemiller117/codecadamy-final-project-data-analysis?scriptVersionId=143149359" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Arabica Coffee Exploration

* [Introduction](#intro)
    * [Background](#background)
    * [Goals](#goals)
* [Data](#data)
    * [Import Python Modules](#libraries)
    * [Understanding the Data](#data_info)
* [Comparing Varities to Cupping Score](#varities_cupping)
    * [Clean Data](#var_clean_data)
    * [Observations](#var_obser)
* [Comparing Altitude to Total Cup Score](#alt_score)
    * [Clean Data](#alt_clean_data)
    * [Examining Data](#alt_score_eda)

<a class="anchor" id="intro"></a>
## Introduction 

<a class="anchor" id="background"></a>
### Background

In the past few years, I have become more interested in coffee. I have thought about how Q-grade affects the price of coffee. However, a more foundational questions deals with how a coffee will score first before it is priced. A Q-grader evaluates the coffee during a process called cupping. The cupping process allows the grader to compare different coffees together. They brew each coffee with the same amount of coffee and water. Specialty Coffee Assocation (SCA) provides the [protocols](https://sca.coffee/research/protocols-best-practices) for cupping an evaluating coffee. Coffees are graded on "Fragrance/Aroma, Flavor, Aftertaste, Acidity, Body, Balance, Uniformity, Clean Cup, Sweetness, Defects, and Overall" (Source: SCA). The graders assign a number between 6-10 using quarter point incriments.

<a class="anchor" id="goals"></a>
### Goals 

I want to discover the connection between cupping score and alititude and also discover if variety affects the score at different alititudes. Here is the order of questions to discover or connect.

1. Clean data as needed for analysis

2. Compare `Total.Cup.Points` with `variety`

3. Compare `Total.Cup.Points` with `altitude_mean_meters`

4. Determine if `altitude_mean_meters` and `variety` effect `Total.Cup.Points`.

<a class="anchor" id="data"></a>
## Data

I have choosen to use the clean data from the data set uploaded to [Kaggle](https://www.kaggle.com/datasets/volpatto/coffee-quality-database-from-cqi). This data comes from James LeDoux [GitHub](https://github.com/jldbc/coffee-quality-database/tree/master/data) repositiary. He scrapped the data from Coffee Quality Institue [database](https://database.coffeeinstitute.org/) in January 2018 then cleaned the data.

<a class="anchor" id="libraries"></a>
### Import Python Modules

First, I imported all libraries required to analysis the data. It also includes Kaggle required os library to access the files. Settings for Pandas and Matplotlib.Pyplot added to format them.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import re

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt # create graphs
import seaborn as sns # create graphs
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
pd.set_option('display.max_columns', None)
plt.style.use('ggplot')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a class="anchor" id="data_info"></a>
### Understanding the Data

The Arabica coffee data contains 44 columns and 1311 rows. Of the 44 columns, we are only interested in a small number of columns, which are `Varitey`, `Total.Cup.Points`, and `altitude_mean_meters`. I used `altitude_mean_meters` instead of `Altitude` because there is clean up. Looking at the head, `Altitude` contains a hyphen and a letter which will have to be removed. The `Altitude` range matched the `altitude_low_meters` and `altitude_high_meters` columns. Looking at total NULL values, `altitude_mean_meters` only contain 4 more NULL values than `Altitude`. To conclude, it made more sense to use the `altitude_mean_meters` thought it will require more clean up. However, `unit_of_measure` contains two values: meters and feet. Since most of the records are meters, I will examine further if we need to clean heights. The columns already have the correct dtypes so there is no need to clean them.

In [ ]:
coffee_df = pd.read_csv("/kaggle/input/coffee-quality-database-from-cqi/arabica_data_cleaned.csv", index_col=0)

print(coffee_df.shape)

In [ ]:
coffee_df.head()

In [ ]:
coffee_df.columns

In [ ]:
coffee_df.info()

In [ ]:
coffee_df.describe()

In [ ]:
coffee_df.isnull().sum()

<a class="anchor" id="varities_cupping"></a>
## Comparing Varities to Cupping Score

<a class="anchor" id="var_clean_data"></a>
### Cleaning Data

In order to anwser the first question, I copied `coffee_df` to keep `Variety` and `Total.Cup.Points`. From above, I noticed that `Total.Cup.Points` has at least one 0 value. I removed those entries since would mean.  

There are also null values for varieties. There are some varieties classified as other. Both values seem to indicate the same issue, they did not know the variety so some say other and some left it blank. Some countries, like Ethiopia, have heirloom coffees where varietial is unknown. 

There are some varieties than have a few entires so all varieties with less than 10 entries were removed. Next, I grouped all vareties with their average cupping scores. Finally, I used a boxplot to graph these varieties and arranged them from highest average cup score descending.

After plotting the boxplot, there were small amount of outliners beyond 70. Before removing the 5 point, the mean was 82.18. The mean raised to 82.26. I ploted the mean on the histograph to notice that the points are skrewed to the right. 

In [ ]:
# Coping the needed columns: Varieties and total cup points
grade_variety = coffee_df[['Variety', 'Total.Cup.Points']].copy().rename(columns={'Total.Cup.Points': 'Total_Cup_Points'})

# Removing any Total Cup Points that equal zero.
grade_variety = grade_variety[grade_variety.Total_Cup_Points > 0].reset_index(drop=True).copy()

# Renaming the NULL values in Variety to "Other"
grade_variety.Variety = grade_variety.Variety.fillna('Other')

# Printing new minimium score
grade_variety.Total_Cup_Points.min()


In [ ]:
variety_count = grade_variety['Total_Cup_Points'].groupby(grade_variety['Variety']).count().sort_values(ascending=False).copy()

variety_count

In [ ]:
variety_count.plot(kind='bar', figsize=[20,15], title='Total Samples of each Variety', ylabel='Number of Records')
plt.axline(xy1=(0,10), slope=0, color='blue')
plt.xticks(ha='right', rotation=20)
plt.show()

In [ ]:
plt.figure(figsize=[20,15], facecolor='white')
sns.boxplot(x='Total_Cup_Points',data=grade_variety)
plt.title('Boxplot of Total Score')
plt.xlabel('Total Cup Score')
plt.show()

The following histograph shows the distrubtion of scores. 

In [ ]:
grade_variety = grade_variety[grade_variety['Total_Cup_Points']>=70].reset_index(drop=True).copy()

In [ ]:
# Calculate grade mean
grade_mean = grade_variety.Total_Cup_Points.mean()
print("The mean for Total Cup Point is " + str(round(grade_mean,2)))
grade_median = grade_variety.Total_Cup_Points.median()
print("The median for Total Cup Point is " + str(round(grade_median,2)))
grade_std = grade_variety.Total_Cup_Points.std()
print("The standard deviation for Total Cup Point is " + str(round(grade_std,2)) + " points")





In [ ]:
plt.figure(figsize=[20,15], facecolor='white')
grade_variety['Total_Cup_Points'].plot(kind='hist', bins=50, title='Number of Coffees per Score > 70', xlabel='Total Cup Score')
plt.axvline(x=grade_mean, color='blue')
plt.show()

In [ ]:
grade_variety['count_variety'] = grade_variety.groupby('Variety')['Total_Cup_Points'].transform('nunique')
grade_df = grade_variety[grade_variety.count_variety >= 10]
grouped_grade_variety = grade_df.loc[:,['Variety', 'Total_Cup_Points']].groupby('Variety').median().sort_values(by='Total_Cup_Points', ascending=False)

In [ ]:
plt.figure(figsize=[20,15], facecolor='white')
sns.boxplot(x='Variety', y='Total_Cup_Points',data=grade_variety, order=grouped_grade_variety.index)
plt.title('Grade by Variety')
plt.xlabel('Variety')
plt.xticks(rotation = 45, ha='right')
plt.ylabel('Total Grade')
#plt.ylim(60, 95)
plt.savefig('grade_by_variety.png')

<a class="anchor" id="var_obser"></a>
#### Observations

The box plot shows that all varieties have a similar average cup total. Gesha is known for being one of the best coffees yet its cupping score does not reflect the high qualtity many give it. The top 3 cupping scores are coffees found in east Africa, specificly Kenya, Malawi, and Zimbabwe. Caturra breaks the chain as a primarly South American coffee varietal. The large amount of outliers show how much variance a coffee variety has. This speaks to the organic nature of coffee. 

<a class="anchor" id="alt_score"></a>
### Comparing Altitude to Total Cup Score

<a class="anchor" id="alt_clean_data"></a>
#### Cleaning Data

First, I looked at the `altitude_mean_meters` and compared to `altitude` given. I started looking at which units were present in the dataframe. The data contains 111 records where the unit is ft. I check unique values for the `Altitude`. Since the column contained characters, I removed the characters leaving numbers. I added a new column to check `Altitude` to `altitude_mean_meters`. Since the columns matched, I can use `altitude_mean_meters` knowing everything is in meters.

In [ ]:
uom = coffee_df[['unit_of_measurement','altitude_mean_meters']].groupby(['unit_of_measurement']).count()
uom

In [ ]:
alt_ft = coffee_df[(coffee_df['unit_of_measurement'] == 'ft')*(coffee_df['altitude_mean_meters'] > 0)].copy()
print(alt_ft['Altitude'].unique())

In [ ]:
alt_ft['Altitude'] = alt_ft['Altitude'].str.extract(pat='(\d+)', expand=False).astype(float)
alt_ft['ft_to_m'] = alt_ft['Altitude'] * 0.3048
print(alt_ft[['Altitude','altitude_mean_meters','ft_to_m']].to_string())


In [ ]:
coffee_df[(coffee_df.altitude_mean_meters < 200)*(coffee_df.altitude_mean_meters >= 100)]

In [ ]:
altitude_df = coffee_df[['altitude_mean_meters', 'Total.Cup.Points']].copy().rename(columns={'Total.Cup.Points': 'Total_Cup_Points'})
altitude_df.altitude_mean_meters = altitude_df.altitude_mean_meters.fillna(0).astype('float64')
altitude_df = altitude_df[altitude_df.Total_Cup_Points > 0].reset_index(drop=True).copy()
altitude_df = altitude_df[altitude_df.altitude_mean_meters > 0].reset_index(drop=True).copy()
altitude_df[altitude_df.altitude_mean_meters < 200]

In [ ]:
altitude_edit_df = altitude_df[altitude_df['altitude_mean_meters'].between(500,2500)]

<a class="anchor" id=alt_score_eda></a>
#### Examining Data

First, I did a simple linear regression between altitude and Total Cup Score. It seems that in general, cup score improves as the coffee is harvested at higher altitudes. 


In [ ]:
grade_variety_alt = coffee_df[['Variety', 'altitude_mean_meters', 'Total.Cup.Points']].copy().rename(columns={'Total.Cup.Points': 'Total_Cup_Points'})
grade_variety_alt.Variety = grade_variety_alt.Variety.fillna('Unknown')
grade_variety_alt.altitude_mean_meters = grade_variety_alt.altitude_mean_meters.fillna(0).astype('float64')
grade_variety_alt = grade_variety_alt[grade_variety_alt['altitude_mean_meters'].between(500,2500)]
print(grade_variety_alt)

In [ ]:
plt.figure(figsize=[20,15], facecolor='white')
sns.regplot(x='altitude_mean_meters',y='Total_Cup_Points', data=altitude_edit_df)
plt.title('Grade by Altitude')
plt.xlabel('Altitude (MSL)')
plt.xticks(rotation = 45)
plt.ylabel('Total Grade')
plt.ylim(60, 95)
plt.xlim(500, 2600)
plt.savefig('grade_by_altitude.png')

In [ ]:
bourbon_cup_score = grade_variety['Total_Cup_Points'][grade_variety.Variety == 'Bourbon']
typica_cup_score = grade_variety['Total_Cup_Points'][grade_variety.Variety == 'Typica']
caturra_cup_score = grade_variety['Total_Cup_Points'][grade_variety.Variety == 'Caturra']
catuai_cup_score = grade_variety['Total_Cup_Points'][grade_variety.Variety == 'Catuai']

In [ ]:
sns.histplot(bourbon_cup_score,color='blue', alpha=.5, label='Bourbon')
sns.histplot(typica_cup_score,color='green', alpha=.5, label='Typica')
sns.histplot(caturra_cup_score,color='red', alpha=.5, label='Caturra')
sns.histplot(catuai_cup_score,color='yellow', alpha=.8, label='Catuai')
plt.legend()
plt.xlim(60, 95)

In [ ]:
ratio_1 = np.std(bourbon_cup_score)/np.std(typica_cup_score)
ratio_2 = np.std(bourbon_cup_score)/np.std(caturra_cup_score)
ratio_3 = np.std(bourbon_cup_score)/np.std(catuai_cup_score)
ratio_4 = np.std(typica_cup_score)/np.std(caturra_cup_score)
ratio_5 = np.std(typica_cup_score)/np.std(catuai_cup_score)
ratio_6 = np.std(caturra_cup_score)/np.std(catuai_cup_score)
print("Bourbon over Typica: " + str(ratio_1))
print("Bourbon over Caturra: " + str(ratio_2))
print("Bourbon over Catuai: " + str(ratio_3))
print("Typica over Caturra: " + str(ratio_4))
print("Typica over Catuai: " + str(ratio_5))
print("Caturra over Catuai: " + str(ratio_6))

The ratios show that only Bourbon over Typica are similar enough. Caturra is quite a bit different from all the other coffees.

In [ ]:
fstat, pval = f_oneway(bourbon_cup_score, typica_cup_score, catuai_cup_score, caturra_cup_score)
print(pval)

Spliting Altitude of varital into bins

In [ ]:
top_score = grade_variety_alt[(grade_variety_alt.Variety == 'Bourbon')|(grade_variety_alt.Variety == 'Typica')|(grade_variety_alt.Variety == 'Caturra')|(grade_variety_alt.Variety == 'Catuai')].copy().reset_index()
print(top_score)

In [ ]:
tukey = pairwise_tukeyhsd(top_score['Total_Cup_Points'], top_score['Variety'], 0.05)
print(tukey)

In [ ]:
top_score['altitude_group'] = pd.qcut(top_score['altitude_mean_meters'],4)
print(top_score)

In [ ]:
plt.figure(figsize=[20,15], facecolor='white')
sns.violinplot(data=top_score,x='Variety',y='Total_Cup_Points',hue='altitude_group', alpha=.5, cut=0)
plt.ylim(60, 95)

In [ ]:
altitude_group = grade_variety_alt[grade_variety_alt["Total_Cup_Points"] >= 1].copy()
altitude_group['altitude_bin'] = pd.qcut(grade_variety_alt['altitude_mean_meters'],4).copy()
altitude_group['score_bin'] = pd.cut(grade_variety_alt['Total_Cup_Points'],[0, 80, 85, 90,100], labels=['Below Specialty Quality','Very Good','Excellent','Outstanding']).copy()
altitude_group

In [ ]:
from scipy.stats import chi2_contingency
score_alt_freq = pd.crosstab(altitude_group['score_bin'], altitude_group['altitude_bin'])
score_alt_freq

In [ ]:
score_alt_prop = score_alt_freq/len(altitude_group)
score_alt_prop

In [ ]:
chi2, pval, dof, expected = chi2_contingency(score_alt_freq)
print(chi2)
print(expected)

In [ ]:
print(altitude_group['score_bin'])